In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
list_file = 'experiments_list.txt'
with open(list_file, 'r') as f:
    dirs = [f'experiments/{line.strip()}' for line in f if line.strip() and not line.strip().startswith('#')]

In [3]:
# Load logs.csv from each directory into a dictionary of DataFrames
dfs = {}
for d in dirs:
    csv_path = os.path.join(d, 'logs.csv')
    if os.path.isfile(csv_path):
        dfs[d] = pd.read_csv(csv_path)
    else:
        print(f'Warning: {csv_path} not found, skipping.')

# Optionally, concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(
    dfs.values(), 
    keys=dfs.keys(), 
    names=['experiment', 'row']
).reset_index(level='row', drop=True).reset_index().rename(columns={'index': 'experiment'})

# Display the combined DataFrame
df = combined_df.query('itr>0')[['experiment','iteration_time']].copy().groupby('experiment').mean().reset_index()

In [4]:
parts = df.experiment.str.split('/', expand=True)
df['Experiment'] = parts[1].copy()
df['Method'] = parts[2].copy()
df['Loss'] = parts[3].copy()

In [6]:
df['Loss'].replace(
    {'adjoint_matching': 'Adjoint Matching',
     'log_variance': 'Log variance',
     'rel_entropy': 'Relative entropy',
     'pinn_GELU': 'PINN',
     'rel_GELU': 'Relative loss',
     'ritz_GELU': 'Deep ritz loss',
     'var_GELU': 'Variational loss'},
     inplace=True
)

df['Experiment'].replace(
    {'OU_anisotropic_d20': 'AQ',
     'OU_stable_d20': 'IQ',
     'OU_hard_d20': 'RQ',
     'double_well_d10': 'DW'},
     inplace=True
)

In [7]:
pivot = df.pivot_table(
    index=['Method', 'Loss'],
    columns='Experiment',
    values='iteration_time',
    aggfunc='mean'
)

In [8]:
print(pivot)

Experiment                       RQ
Method   Loss                      
COMBINED Adjoint Matching  0.331509
         Log variance      0.328471
         Relative entropy  0.418798
         SOCM              0.432274
EIGF     Deep ritz loss    0.227275
         PINN              0.661866
         Relative loss     0.661570
         Variational loss  0.228154
FBSDE    FBSDE             0.443269
IDO      Adjoint Matching  0.229828
         Log variance      0.212383
         Relative entropy  0.413282
         SOCM              0.798727


In [9]:
latex_str = pivot.to_latex(
    caption='Iteration times by experiment, method and loss',
    label='tab:iteration_times',
    escape=True,            # <-- ensure special chars (_ & %) get backslash-escaped
    multicolumn=True,
    multicolumn_format='c',
    float_format="%.3f",
    bold_rows=True
)
print(latex_str)

\begin{table}
\caption{Iteration times by experiment, method and loss}
\label{tab:iteration_times}
\begin{tabular}{llr}
\toprule
 & Experiment & RQ \\
Method & Loss &  \\
\midrule
\multirow[t]{4}{*}{\textbf{COMBINED}} & \textbf{Adjoint Matching} & 0.332 \\
\textbf{} & \textbf{Log variance} & 0.328 \\
\textbf{} & \textbf{Relative entropy} & 0.419 \\
\textbf{} & \textbf{SOCM} & 0.432 \\
\cline{1-3}
\multirow[t]{4}{*}{\textbf{EIGF}} & \textbf{Deep ritz loss} & 0.227 \\
\textbf{} & \textbf{PINN} & 0.662 \\
\textbf{} & \textbf{Relative loss} & 0.662 \\
\textbf{} & \textbf{Variational loss} & 0.228 \\
\cline{1-3}
\textbf{FBSDE} & \textbf{FBSDE} & 0.443 \\
\cline{1-3}
\multirow[t]{4}{*}{\textbf{IDO}} & \textbf{Adjoint Matching} & 0.230 \\
\textbf{} & \textbf{Log variance} & 0.212 \\
\textbf{} & \textbf{Relative entropy} & 0.413 \\
\textbf{} & \textbf{SOCM} & 0.799 \\
\cline{1-3}
\bottomrule
\end{tabular}
\end{table}

